In [391]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['UTILS_PATH'] = r'C:\Users\Mi\Desktop\code\git\python-utils-ak'
os.environ['PROJECT_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat' 
os.environ['SQLITE_PATH'] = r'C:\Users\Mi\Desktop\code\git\2020.10-umalat\umalat\data.sqlite' 
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [404]:
root = make_schedule(boiling_plan_df)
# wb = draw_workbook(root, mode='prod')
# wb.save('output.xlsx')
# open_file_in_os('output.xlsx') 

In [62]:
pyperclip.copy(draw_print(root, visible_only=True))

In [ ]:
style = load_style(mode='prod')
root.props.update({'size': max(c.end for c in root.children)})
init_sheet_func = init_sheet if mode == 'dev' else init_template_sheet(template_fn=template_fn)
draw_schedule(root, style, init_sheet_func=init_sheet_func)

# Drawing

In [445]:
boiling_plan_df = pd.read_csv('2020.12.13 boiling_plan.csv')
boiling_plan_df['sku'] = boiling_plan_df['sku'].apply(cast_sku)
# boiling_plan_df['boiling'] = boiling_plan_df['boiling'].apply(cast_boiling)
boiling_plan_df['boiling'] = boiling_plan_df['sku'].apply(lambda sku: sku.boilings[0])
boiling_plan_df.sort_values(by='id')

def add_push_with_props(parent, block):
    return add_push(parent, block, new_props=block.props.get_all_props())

mode = 'prod'
maker = BlockMaker(default_push_func=dummy_push)
make = maker.make

root = make_schedule(boiling_plan_df)

In [446]:
with make('schedule'):
    # todo: make properly
    make(orient='vertical', h=1, visible=False)
    with make('header', orient='vertical', h=1, index_width=3):
        make(h=1, size=1, text='График наливов', push_func=add_push)
        for i in range(288):
            cur_time = cast_time(i + cast_t('01:00'))
            if cur_time[-2:] == '00':
                make(t=1 + i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=1 + i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    for i in range(2):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)

    with make('cleanings', orient='vertical', h=2):
        for block in root['cleaning']:
            make(block, push_func=add_push_with_props)
    make(orient='vertical', h=1, visible=False)

    for i in range(2, 4):
        with make(f'cheese_maker_{i}', orient='vertical', h=2):
            for block in root['boiling']:
                if block.props['pouring_line'] == str(i):
                    make(block['pouring'], push_func=add_push_with_props)
        make(orient='vertical', h=1, visible=False)
    
    with make('header', orient='vertical', h=1, index_width=4):
        for i in range(288):
            cur_time = cast_time(i + cast_t('07:00'))
            if cur_time[-2:] == '00':
                make(t=i, size=1, h=1, text=str(int(cur_time[:2])), color=(218, 150, 148), text_rotation=90, push_func=add_push)
            else:
                make(t=i, size=1, h=1, text=cur_time[-2:], color=(204, 255, 255), text_rotation=90, push_func=add_push)
    
    with make('water_melting', orient='vertical', h=4):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['melting'], push_func=add_push_with_props)
                
    with make('water_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'water':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
    
    # todo: make dynamic lines
    n_lines = 5
    melting_lines = [make(f'salt_melting_{i}', orient='vertical', h=3).block for i in range(n_lines)]
    
    # todo: hardcode, add empty elements for drawing not to draw melting_line itself
    for b in melting_lines:
        add_push(b, Block('block', visible=False))
        
    for i, block in enumerate([b for b in root['boiling'] if b.props['boiling_type'] == 'salt']):
        add_push_with_props(melting_lines[i % n_lines], block['melting_and_packing']['melting'])
    
    with make('water_packing', orient='vertical', h=3):
        for block in root['boiling']:
            if block.props['boiling_type'] == 'salt':
                make(block['melting_and_packing']['packing_and_preconfiguration'], push_func=add_push_with_props)
row_root = maker.root

In [447]:
row_root.props.update({'h': max(c.end for c in root.children)})
style = load_style()
init_sheet_func = init_sheet if mode == mode else init_template_sheet(template_fn=template_fn)
wb = draw_schedule(row_root, style, init_sheet_func=init_sheet_func)
wb.save('output.xlsx')
open_file_in_os('output.xlsx') 

In [448]:
3 кг - 360
0.36 - 728

SyntaxError: invalid syntax (<ipython-input-448-fad91589d66b>, line 1)

In [545]:
# one group element can be used at one time
S = 1020

df = pd.DataFrame(columns=['speed', 'qty'])
df['qty'] = [800, 50]
df['speed'] = [728, 360]
df['group'] = ['non-rubber', 'rubber']
df

,speed,qty,group
0,728,800,non-rubber
1,360,50,rubber


In [549]:
remove_duplicates([2,3,1,1, ])

[2, 3, 1]

In [550]:
def get_total_parallel_time(input_speed, df):
    t = 0

    # start with slow ones
    df = df.sort_values(by='speed')
    
    while df['qty'].sum() > 0:
        speed_left = S
        
        groups = remove_duplicates(df['group'])
        group_df = pd.DataFrame([False] * len(groups), index=groups, columns=['used'])
        
        df['cur_speed'] = 0
        
        for group in groups:
            grp = df[df['group'] == group]
            
            # set cur speed
            for i, row in grp.iterrows():
                if row['qty'] == 0:
                    continue
                
                if group_df.at[row['group'], 'used']:
                    break
                
                df.at[i, 'cur_speed'] = min(speed_left, row['speed'])
                speed_left -= df.at[i, 'cur_speed']
                
                group_df.at[row['group'], 'used'] = True
        
        df['time_left'] = df['qty'] / df['cur_speed']
        print(df)
        min_time_left = df['time_left'][df['time_left'] > 0].min()
        t += min_time_left
        df['qty'] -= df['cur_speed'] * min_time_left
        print(df)
        print()
    return t

In [551]:
get_total_parallel_time(S, df)

   speed  qty       group  cur_speed  time_left
1    360   50      rubber        360   0.138889
0    728  800  non-rubber        660   1.212121
   speed         qty       group  cur_speed  time_left
1    360    0.000000      rubber        360   0.138889
0    728  708.333333  non-rubber        660   1.212121

   speed         qty       group  cur_speed  time_left
1    360    0.000000      rubber          0        NaN
0    728  708.333333  non-rubber        728   0.972985
   speed  qty       group  cur_speed  time_left
1    360  0.0      rubber          0        NaN
0    728  0.0  non-rubber        728   0.972985



1.111874236874237